# Demo 03 - Regression Analysis

Up to this point, we've performed basic descriptive analytics, looking at counts.  We can also use regression analysis to help us understand our data better.

In [ ]:
if(!require(tidyverse)) {
    install.packages("tidyverse", repos = "http://cran.us.r-project.org")
    library(tidyverse)
}

if(!require(odbc)) {
    install.packages("odbc", repos = "http://cran.us.r-project.org")
    library(odbc)
}

# ggplot2 is installed with the tidyverse.
library(ggplot2)

## Regressing Expenditures Against Bus Counts

We have data broken down by month to try to avoid some of the noise while still giving us enough data points for regression.

In [ ]:
conn <- DBI::dbConnect(odbc::odbc(), 
                      Driver = "SQL Server", 
                      Server = "localhost", 
                      Database = "ForensicAccounting", 
                      Trusted_Connection = "True")

In [ ]:
expenditures <- DBI::dbGetQuery(conn, "WITH buses AS
(
	SELECT
		c.FirstDayOfMonth,
		c.CalendarMonth,
		c.CalendarYear,
		COUNT(*) AS NumberOfBuses
	FROM dbo.Bus b
		INNER JOIN dbo.Calendar c
			ON b.DateFirstInService <= c.Date
			AND ISNULL(b.DateRetired, '2018-12-31') >= c.Date
	WHERE
		c.Date = c.FirstDayOfMonth
		AND c.CalendarYear >= 2011
		AND c.CalendarYear < 2019
	GROUP BY
		c.FirstDayOfMonth,
		c.CalendarMonth,
		c.CalendarYear
),
expenses AS
(
	SELECT
		c.FirstDayOfMonth,
		COUNT(*) AS NumberOfInvoices,
		SUM(li.Amount) AS TotalInvoicedAmount
	FROM dbo.LineItem li
		INNER JOIN dbo.Calendar c
			ON li.LineItemDate = c.Date
	GROUP BY
	c.FirstDayOfMonth
)
SELECT
	b.FirstDayOfMonth,
	b.CalendarMonth,
	b.CalendarYear,
	b.NumberOfBuses,
	e.NumberOfInvoices,
	e.TotalInvoicedAmount
FROM buses b
	INNER JOIN expenses e
		ON b.FirstDayOfMonth = e.FirstDayOfMonth
ORDER BY
	b.FirstDayOfMonth;
")

In [ ]:
summary(expenditures)
glimpse(expenditures)

The `lm()` function allows us to build a linear regression quickly in R.  Our assumption is that bus expenditures scale linearly with respect to the number of buses in our fleet.  This might not be a perfect understanding of the data but should be good for a first glance.

In [ ]:
reg <- lm(formula = TotalInvoicedAmount ~ NumberOfBuses, data = expenditures)

In [ ]:
summary(reg)

As a first approximation, the number of buses explains about 38% of the variance in data.  This graph can show us why:

In [ ]:
options(repr.plot.width=6, repr.plot.height=4) 
ggplot(data = expenditures, aes(x = NumberOfBuses, y = TotalInvoicedAmount)) +
    geom_point() +
    geom_smooth()

But remember that we saw the data in 2018 spike.  Given that, let's see how the regression works prior to 2018.

In [ ]:
regPre2018 <- lm(formula = TotalInvoicedAmount ~ NumberOfBuses,
                 data = filter(expenditures, lubridate::year(FirstDayOfMonth) < 2018))

In [ ]:
summary(regPre2018)

In [ ]:
options(repr.plot.width=6, repr.plot.height=4) 
plot(regPre2018)

It turns out that this is only a minor improvement.  But let's look at counts.

## Regressing Invoice Counts Against Bus Counts

In [ ]:
regICPre2018 <- lm(formula = NumberOfInvoices ~ NumberOfBuses,
                 data = filter(expenditures, lubridate::year(FirstDayOfMonth) < 2018))

In [ ]:
summary(regICPre2018)

Doing this against the count of invoices gives us an R^2 of 0.45, so it's a moderate regression.  So maybe there's a bit too much noise in the monthly data.

We might also be missing an important variable.  Number of buses is significant according to our probability test---its p-value is tiny.  A small p-value isn't proof of anything but here the result fits our intuition:  more buses means more invoices.  Specifically, we get approximately 1 invoice per bus per month--the estimate is 0.9771, so I'm rounding it for purposes of understanding and because these estimates have error bars, meaning it's not exactly 0.9771 but some range around 0.9771.

That aside, let's see if we can do better here.  The three variables we have in our SQL query are number of buses, calendar month and calendar year.  Let's see how adding the calendar dates change our results.  First we'll add month.

In [ ]:
regICPre2018 <- lm(formula = NumberOfInvoices ~ NumberOfBuses + CalendarMonth,
                 data = filter(expenditures, lubridate::year(FirstDayOfMonth) < 2018))

In [ ]:
summary(regICPre2018)

There is a **very** small change in the R^2 from 45% to 46% of variance explained.  Not only is that nothing to write home about, it's indistinguishable from noise.

How about if we use calendar year instead?

In [ ]:
regICPre2018 <- lm(formula = NumberOfInvoices ~ NumberOfBuses + CalendarYear,
                 data = filter(expenditures, lubridate::year(FirstDayOfMonth) < 2018))

In [ ]:
summary(regICPre2018)

Now this result is quite interesting.  Our R^2 didn't change but now neither variable is significant!  This is a great example of something called multicollinearity, one of the challenges of regression.  Put simply, the number of buses increases by about the same number every year, so there is very high correlation between number of buses and calendar year:

In [ ]:
exp2017 <- filter(expenditures, lubridate::year(FirstDayOfMonth) < 2018)
cor(exp2017$NumberOfBuses, exp2017$CalendarYear)

That is, 99.6% of the variance reflected in buses is also reflected in year.  These two variables are **co-linear**.  Because these two variables move almost 1 for 1, it is difficult for the regression algorithm to separate behavior in one versus the other.  They're both fighting to explain the same variance and so both end up with higher p-values.  Also of interest is that the R^2 doesn't change.  Multicollinearity doesn't make your overall predictions worse, but it does make it tougher to tell which independent variables are driving the change.

This is an extreme scenario, mind you, but mutlicollinearity is a common enough occurrence that you will want to be on the lookout for it.

Now let's take a step back, as we're not getting the job done with regressing at the month level.

## Regressing Annual Data

Let's look at annual data to see if this is a tighter fit.

In [ ]:
annualExpenditures <- DBI::dbGetQuery(conn, "WITH buses AS
(
	SELECT
		c.CalendarYear,
		COUNT(*) AS NumberOfBuses
	FROM dbo.Bus b
		INNER JOIN dbo.Calendar c
			ON b.DateFirstInService <= c.Date
			AND ISNULL(b.DateRetired, '2018-12-31') >= c.Date
	WHERE
		c.CalendarDayOfYear = 1
		AND c.CalendarYear >= 2011
		AND c.CalendarYear < 2019
	GROUP BY
		c.CalendarYear
),
expenses AS
(
	SELECT
		c.CalendarYear,
		COUNT(*) AS NumberOfInvoices,
		SUM(li.Amount) AS TotalInvoicedAmount
	FROM dbo.LineItem li
		INNER JOIN dbo.Calendar c
			ON li.LineItemDate = c.Date
	GROUP BY
	c.CalendarYear
)
SELECT
	b.CalendarYear,
	b.NumberOfBuses,
	e.NumberOfInvoices,
	e.TotalInvoicedAmount
FROM buses b
	INNER JOIN expenses e
		ON b.CalendarYear = e.CalendarYear
ORDER BY
	b.CalendarYear;
")

In [ ]:
regICAnnualPre2018 <- lm(formula = NumberOfInvoices ~ NumberOfBuses,
                 data = filter(annualExpenditures, CalendarYear < 2018))

In [ ]:
summary(regICAnnualPre2018)

In [ ]:
options(repr.plot.width=6, repr.plot.height=4) 
plot(regICAnnualPre2018)

Now we have a 99% R^2.  At the annual level, these differences seem to smooth out.  The number of data points is small so be forewarned but it looks stable.  Now let's look at it with 2018 data in.

In [ ]:
regICAnnual <- lm(formula = NumberOfInvoices ~ NumberOfBuses,
                 data = annualExpenditures)

In [ ]:
summary(regICAnnual)

Note that the R^2 drops from 99% to 46%.  That's a pretty big drop for one year.

One last thing is, I'd like to see what the previous model would have predicted for 2018.  We can use the `predict()` function to perform this operation easily.

In [ ]:
predict(regICAnnualPre2018, newdata = filter(annualExpenditures, CalendarYear == 2018))

In [ ]:
annualExpenditures %>% arrange(desc(CalendarYear))